In [56]:
import torch
import numpy as np
from math import sin, cos

SENTENCE = "I LIKE PIE VERY MUCH PIE ."
DIM_VOCAB = 512
torch.manual_seed(7)

vocab = dict()
i = 0
for token in SENTENCE.split():
    if token not in vocab:
        vocab[token] = i
        i += 1
vocab["<unk>"] = i
vocab["<s>"] = i + 1
print(vocab)

DICT_SIZE = len(vocab)

oneHotSent = np.zeros((len(SENTENCE.split()), DICT_SIZE), dtype=float)

for i in range(len(SENTENCE.split())):
    word = SENTENCE.split()[i]
    oneHotSent[i][vocab[word]] = 1

oneHotTorch = torch.as_tensor(oneHotSent, dtype=torch.float32)
    
#print(oneHotTorch)
#print(oneHotTorch.shape)
#print(oneHotTorch.dtype)

#DATA PREPARATION DONE

We_skeleton = torch.tensor(np.zeros((DICT_SIZE, DIM_VOCAB), dtype=float), dtype=torch.float32,).normal_(0, 1)
We = torch.autograd.Variable(We_skeleton, requires_grad=True)
print(We)
print(We.shape)
print(We.dtype)


Batch = oneHotTorch.mm(We)
Pos_emb = np.zeros(Batch.shape, dtype=float)
for p in range(len(SENTENCE.split())):
    for i in range(DIM_VOCAB//2):
        val = p/(10000**(2*i/DIM_VOCAB))
        Pos_emb[p][2*i] = sin(val)
        Pos_emb[p][2*i + 1] = cos(val)

Pos_emb_var = torch.autograd.Variable(torch.tensor(Pos_emb, dtype=torch.float32), requires_grad=False)
print(Pos_emb_var)

print(Batch)
#Scale batch up
Batch = Batch*(DIM_VOCAB**(1/2))
print(Batch)
Batch = Batch + Pos_emb_var
print(Batch)

print(Batch.shape)
print(Batch.dtype)

#Batch (in this case a one sentence batch) is prepared. Maxi calls this embedded sentence


{'I': 0, 'LIKE': 1, 'PIE': 2, 'VERY': 3, 'MUCH': 4, '.': 5, '<unk>': 6, '<s>': 7}
tensor([[-0.8201,  0.3956,  0.8989,  ...,  0.2500, -0.8542,  0.4376],
        [ 0.3811,  0.3953, -1.5423,  ...,  0.6046,  0.3367, -0.9041],
        [ 0.8511,  0.1124,  1.3421,  ...,  0.1326,  2.4947,  1.4662],
        ...,
        [-0.1571, -0.9080, -1.6175,  ...,  1.7259,  2.4928, -0.9008],
        [ 0.7208,  1.3598, -0.4817,  ..., -1.1913,  0.9311, -1.6440],
        [ 2.7773,  0.2596,  1.1394,  ..., -0.3252, -0.5689, -1.6829]],
       requires_grad=True)
torch.Size([8, 512])
torch.float32
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  8.2186e-01,  ...,  1.0000e+00,
          1.0366e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.3641e-01,  ...,  1.0000e+00,
          2.0733e-04,  1.0000e+00],
        ...,
        [-7.5680e-01, -6.5364e-01, -6.5717e-01,  ...,  1.0000e+00,
          4.1465e-04,  1.0000e+00

In [55]:
class layer:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-55-361f39786ac1>, line 2)